<a href="https://colab.research.google.com/github/elizperezsilva/deep-learning-challenge/blob/main/AlphabetSoupCharityOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Preview the dataset
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop unnecessary columns
application_df = application_df.drop(columns=["EIN", "NAME"])


In [ ]:
# Bin APPLICATION_TYPE
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_counts[application_counts < 1000].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Bin CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 500].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")

# Verify changes
print(application_df['APPLICATION_TYPE'].value_counts())
print(application_df['CLASSIFICATION'].value_counts())


APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64


In [ ]:
# One-hot encode categorical variables
application_df = pd.get_dummies(application_df)

# Split features (X) and target (y)
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values
y = application_df["IS_SUCCESSFUL"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Optimization Attempt #1

In [ ]:
# Define the model
nn = tf.keras.models.Sequential()

# Input layer and first hidden layer
nn.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))  # Explicitly define input shape
nn.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 100)                 │           4,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,301 (36.33 KB)

 Trainable params: 9,301 (36.33 KB)

 Non-trainable params: 0 (0.00 B)

Optimization Attempt #2

In [ ]:
# Define the model
nn = tf.keras.models.Sequential()

# Input layer and first hidden layer with tanh activation
nn.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
nn.add(tf.keras.layers.Dense(units=100, activation="tanh"))

# Second hidden layer with LeakyReLU activation
nn.add(tf.keras.layers.Dense(units=50))
nn.add(tf.keras.layers.LeakyReLU(negative_slope=0.01))  # Replace alpha with negative_slope

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 100)                 │           4,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,301 (36.33 KB)

 Trainable params: 9,301 (36.33 KB)

 Non-trainable params: 0 (0.00 B)

Optimization Attempt #3

In [ ]:
# Train the model with more epochs
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=100,  # Increased epochs
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7050 - loss: 0.5866 - val_accuracy: 0.7363 - val_loss: 0.5524
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7231 - loss: 0.5622 - val_accuracy: 0.7363 - val_loss: 0.5494
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7256 - loss: 0.5584 - val_accuracy: 0.7380 - val_loss: 0.5503
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7309 - loss: 0.5553 - val_accuracy: 0.7298 - val_loss: 0.5501
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7290 - loss: 0.5531 - val_accuracy: 0.7371 - val_loss: 0.5492
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7324 - loss: 0.5511 - val_accuracy: 0.7376 - val_loss: 0.5455
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7297 - loss: 0.5541 - val_accuracy: 0.7380 - val_loss: 0.5466
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7352 - loss: 0.5453 - val_accu

In [ ]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 0s - 2ms/step - accuracy: 0.7277 - loss: 0.5708
Loss: 0.5707876682281494, Accuracy: 0.7276967763900757


Addtional Optimization Attempts

In [ ]:
# Define the model with more layers and neurons
nn = tf.keras.models.Sequential()

# Input layer and first hidden layer
nn.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
nn.add(tf.keras.layers.Dense(units=128, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Compile the model with RMSprop optimizer
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])


In [ ]:
# Define the model with dropout layers
nn = tf.keras.models.Sequential()

# Input layer and first hidden layer
nn.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
nn.add(tf.keras.layers.Dense(units=128, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))  # Dropout layer with 20% dropout

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model for more epochs
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=100,  # Increased epochs
    batch_size=32,
    validation_split=0.2
)


Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6843 - loss: 0.6105 - val_accuracy: 0.7378 - val_loss: 0.5521
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7207 - loss: 0.5700 - val_accuracy: 0.7372 - val_loss: 0.5489
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7253 - loss: 0.5603 - val_accuracy: 0.7338 - val_loss: 0.5525
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7304 - loss: 0.5557 - val_accuracy: 0.7398 - val_loss: 0.5478
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7284 - loss: 0.5578 - val_accuracy: 0.7416 - val_loss: 0.5473
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7252 - loss: 0.5609 - val_accuracy: 0.7391 - val_loss: 0.5483
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7289 - loss: 0.5584 - val_accuracy: 0.7365 - val_loss: 0.5502
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7309 - loss: 0.5519 - val_accu

In [ ]:
# Evaluate the optimized model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 0s - 1ms/step - accuracy: 0.7278 - loss: 0.5620
Loss: 0.5620462894439697, Accuracy: 0.7278425693511963


In [ ]:
# Save the optimized model
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity_Optimization.h5')
files.download('/content/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>